In [68]:
import numpy as np 
import pandas as pd 
import portion as P
import itertools
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable 
from math import prod


# Transforming LCL:s to linear problems

Goal is to find $\alpha, \beta, a,b,c...$ such that if some $A, A, B$ is possible for active and/or passive nodes, then the corresponding $a+a+b$ will be $\geq \alpha$ and/or $\leq \beta $. Additionally it would be great to see if these values $a, b, c...$ could be expanded to intervals.

## Notes:
It seems to be difficult to find problems that can be expressed in this way, as the constraints for active and passive nodes are unbounded in the other direction. This means that when we have an absolute order for our labels, active (passive) nodes must always allow strengthening (weakening) of labels, as the sum will always be larger (smaller) and thus complying with the constraints. The current version tries to find ranges $\alpha, \beta \subseteq [0, \max(d,\delta)]$, so that sum of edges incident to active (passive) nodes must be within the range $\alpha \; (\beta)$.

## Example problem:

```
A AB AB

B AB AB
```
can be interpreted as $\alpha = 1, \beta=2, A = \left[ 0,\frac{1}{3}\right), B =  \left(\frac{2}{3}, 1\right]$.

What is the system of linear inequalities that could be used to derive this solution? Try:

\begin{align*}
3a &\leq \beta \\
2a+b &\leq \beta \\
a+2b &\leq \beta \\
3b &> \beta \\
\\
3a &< \alpha \\
2a+b &\geq \alpha \\
a+2b &\geq \alpha \\
3b &\geq \alpha

\end{align*}

In [69]:
# Initialize variables

# Add active and passive constraints in RE-formalism
active = """
D ABCD ABCD
C BC ABC
"""
passive = """
A ABCD ABCD
B BC BCD
"""

# Enable/disable debug mode
debug = True

In [70]:
# Create table of possible neighbourhoods and whether they are suitable for active/passive nodes

actives = list(map(lambda x: x.split(), active.strip().split("\n")))
passives = list(map(lambda x: x.split(), passive.strip().split("\n")))

# max(d, delta)
sum_max = max(len(actives[0]), len(passives[0]))

variables = sorted(list(set(list("".join("".join([active, passive]).split())))))

combinations = pd.DataFrame({"combination": itertools.combinations_with_replacement(variables, len(passives[0]))})
combinations["active"] = False
combinations["passive"] = False

for row in passives:
    for c in map(lambda c: tuple(sorted(c)), itertools.product(*row)):
        combinations.loc[combinations["combination"]==c, "passive"] = True 

for row in actives:
    for c in map(lambda c: tuple(sorted(c)), itertools.product(*row)):
        combinations.loc[combinations["combination"]==c, "active"] = True 

if debug: print(combinations)


   combination  active  passive
0    (A, A, A)   False     True
1    (A, A, B)   False     True
2    (A, A, C)   False     True
3    (A, A, D)    True     True
4    (A, B, B)   False     True
5    (A, B, C)    True     True
6    (A, B, D)    True     True
7    (A, C, C)    True     True
8    (A, C, D)    True     True
9    (A, D, D)    True     True
10   (B, B, B)   False     True
11   (B, B, C)    True     True
12   (B, B, D)    True     True
13   (B, C, C)    True     True
14   (B, C, D)    True     True
15   (B, D, D)    True    False
16   (C, C, C)    True    False
17   (C, C, D)    True    False
18   (C, D, D)    True    False
19   (D, D, D)    True    False


In [71]:

epsilon = 0.001

model = LpProblem(name="Reductions", sense=LpMaximize)

alpha_0 = LpVariable(name = "alpha_0", lowBound=0, upBound=sum_max)
alpha_1 = LpVariable(name = "alpha_1", lowBound=0, upBound=sum_max)

beta_0 = LpVariable(name = "beta_0", lowBound=0, upBound=sum_max)
beta_1 = LpVariable(name = "beta_1", lowBound=0, upBound=sum_max)

pulp_variables = dict(zip(variables, [LpVariable(name = v, lowBound=0, upBound=1) for v in variables]))

# Try different targets in order to get more readable solutions
model += beta_0-alpha_0

# Add constraints
trick_variables = []
for index, row in combinations.iterrows():
    if row["active"]:
        model += (sum(pulp_variables[c] for c in row["combination"])>=alpha_0, f"{index}_Active_low")
        model += (sum(pulp_variables[c] for c in row["combination"])<=alpha_1, f"{index}_Active_high")
    else:
        # Sum not in [alpha_0, alpha_1] is the same as sum<alpha_0 OR sum>alpha_1
        # Trick to implement OR in linear programming from https://download.aimms.com/aimms/download/manuals/AIMMS3OM_IntegerProgrammingTricks.pdf
        trick_variables.append(LpVariable(name = f"trick_{index}_a", lowBound=0, upBound=1, cat='Binary'))
        model += (sum(pulp_variables[c] for c in row["combination"])>= alpha_1+epsilon - 1000000*trick_variables[-1], f"{index}_not_Active_high")
        model += (sum(pulp_variables[c] for c in row["combination"])<= alpha_0-epsilon + 1000000*(1-trick_variables[-1]), f"{index}_not_Active_low")

    if row["passive"]:
        model += (sum(pulp_variables[c] for c in row["combination"])>=beta_0, f"{index}_Passive_low")
        model += (sum(pulp_variables[c] for c in row["combination"])<=beta_1, f"{index}_Passive_high")
    else:
        trick_variables.append(LpVariable(name = f"trick_{index}_p", lowBound=0, upBound=1, cat='Binary'))
        model += (sum(pulp_variables[c] for c in row["combination"])>= beta_1+epsilon - 1000000*trick_variables[-1], f"{index}_not_Passive_high")
        model += (sum(pulp_variables[c] for c in row["combination"])<= beta_0-epsilon + 1000000*(1-trick_variables[-1]), f"{index}_not_Passive_low")
    

    
if debug: print(model)

if model.solve() == 1:
    print("Linear model found:")

    for var in model.variables():
        if "trick" not in var.name or debug:
            print(var, var.value())

else: print("No suitable model found")

Reductions:
MAXIMIZE
-1*alpha_0 + 1*beta_0 + 0
SUBJECT TO
0_not_Active_high: 3 A - alpha_1 + 1000000 trick_0_a >= 0.001

0_not_Active_low: 3 A - alpha_0 + 1000000 trick_0_a <= 999999.999

0_Passive_low: 3 A - beta_0 >= 0

0_Passive_high: 3 A - beta_1 <= 0

1_not_Active_high: 2 A + B - alpha_1 + 1000000 trick_1_a >= 0.001

1_not_Active_low: 2 A + B - alpha_0 + 1000000 trick_1_a <= 999999.999

1_Passive_low: 2 A + B - beta_0 >= 0

1_Passive_high: 2 A + B - beta_1 <= 0

2_not_Active_high: 2 A + C - alpha_1 + 1000000 trick_2_a >= 0.001

2_not_Active_low: 2 A + C - alpha_0 + 1000000 trick_2_a <= 999999.999

2_Passive_low: 2 A + C - beta_0 >= 0

2_Passive_high: 2 A + C - beta_1 <= 0

3_Active_low: 2 A + D - alpha_0 >= 0

3_Active_high: 2 A + D - alpha_1 <= 0

3_Passive_low: 2 A + D - beta_0 >= 0

3_Passive_high: 2 A + D - beta_1 <= 0

4_not_Active_high: A + 2 B - alpha_1 + 1000000 trick_4_a >= 0.001

4_not_Active_low: A + 2 B - alpha_0 + 1000000 trick_4_a <= 999999.999

4_Passive_low: A + 2 